# Jupyter_notebook_user_interface-Aatheethyaa dhanasekaran(5051520)

* This node helps to control and monitors an holomonic robot in gazebo environment.
## Description of the user interface
* the user interface is created for better understanding and for better visualization using jupyter notebook and the code can be run using the command rt2_assignment1.launch.
* Here matplot libraries and ipy widgets are used to make the visualization better.
## Features added to user_interface:
* start,stop,forward,backward,right_turn,left_turn buttons are created
* A map to show the moving robot's position
* A histogram plot to show the time to reach a goal by the robot
* A bar graph to to visualize the number of reached targets and cancelled targets
* jupyros is used for interacting with the ros.
* action libraries are used for interacting with ros actions.

In [1]:
%matplotlib widget
import numpy as npy
import collections
import jupyros as jr
import rospy
import time
import actionlib
import actionlib.msg
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from rosgraph_msgs.msg import Clock
from nav_msgs.msg import Odometry
import matplotlib.pyplot as plot
from matplotlib import animation, rc
import matplotlib.ticker as ticker
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox ,Box ,AppLayout , GridspecLayout
import ipywidgets as widgets

#Control node initialization
rospy.init_node('jupyter_rt2_node')


# random behaviour variable
global random_moving

task_time=0
time_updation=[]
reached_targets = 0
cancelled_targets = 0
setting_linear = 1.0
setting_angular = 1.0


# BUTTONS TO CONTROL
* Buttons to control the holomonic robot in the gazebo environment
* six buttons are provided in this block to manually control the robot. The buttons are used to define robot;s direction and it is publishing to the topic /cmd_vel.
* A start button used to trigger the "go_to_point" random movement of the robot and a stop button is provided to set all the robot velocities to zero. The buttons are implement using a service client that send a "start"/"stop" string to the "Command" service.
* Also,there are forward,backward,right and left buttons which are used to move the robot in a required direction.


In [2]:
start  = Button(description='Start',layout=Layout(width='auto', align="center", grid_area='Start'),style=ButtonStyle(button_color='green'))
stop  = Button(description='Stop',layout=Layout(width='auto', grid_area='Stop'),style=ButtonStyle(button_color='red'))
forward  = Button(description='Forward',layout=Layout(width='auto', align="center", grid_area='W'),style=ButtonStyle(button_color='cyan'))
left  = Button(description='Left',layout=Layout(width='auto', grid_area='left'),style=ButtonStyle(button_color='cyan'))
right = Button(description='Right',layout=Layout(width='auto', grid_area='right'),style=ButtonStyle(button_color='cyan'))
backward = Button(description='Backward',layout=Layout(width='auto', grid_area='backward'),style=ButtonStyle(button_color='cyan'))
output = widgets.Output()

#Node initialization
ui_client = rospy.ServiceProxy('/user_interface', Command)
action_service = actionlib.SimpleActionServer(
        '/go_to_point', rt2_assignment1.msg.PlanningAction, auto_start=False)
action_client = actionlib.SimpleActionClient('/go_to_point', rt2_assignment1.msg.PlanningAction)



# Stop the robot and initialize its status
ui_client("stop")
random_moving = False



# funcion o buttons
def start_robot(start):
    global random_moving,task_time,ui_client
    with output:
        setting_linear = 1.0
        setting_angular = 1.0
    ui_client("start")
    random_moving=True
    task_time=rospy.Time.now()
    print('Start is been activated and robot is moving')
        

def stop_robot(stop):
    global random_moving,cancelled_targets
    with output:
        ui_client("stop")
        if random_moving == False:
            print("I was already stopped")
        random_moving = False 
        cancelled_targets=cancelled_targets+1
        task_time=0
        print('Stop is activated and robot will be stopped')
        
def move_forward(forward):
    global random_moving, setting_linear, setting_linear,cancelled_targets
    if random_moving == True:
        ui_client("stop")
        task_time=0
        cancelled_targets=cancelled_targets+1
    else:   
        random_moving = False
        msg.linear.x = setting_linear
        msg.angular.z = 0.0
        pub.publish(msg)
        print("Robot will go straight after the command")     

def move_backward(backward):
    global random_moving, setting_linear, setting_angular,cancelled_targets
    if random_moving == True:
        ui_client("stop")
        task_time=0
        cancelled_targets=cancelled_targets+1
    else:
        random_moving = False
        msg.linear.x = -setting_linear
        msg.angular.z = setting_angular
        setting_linear = msg.linear.x 
        setting_angular = msg.angular.z 
        pub.publish(msg)
        print("Robot will go backward after the command")
        
def turn_right(right):
    global random_moving, setting_linear, setting_angular,cancelled_targets
    if random_moving == True:
        ui_client("stop")
        cancelled_targets=cancelled_targets+1
        task_time=0
    else:
        random_moving = False
        msg.linear.x = setting_linear
        msg.angular.z = setting_angular
        pub.publish(msg)
        print("Robot will go right-side after the command")
        
def turn_left(left):
    global random_moving, setting_linear, setting_angular,cancelled_targets
    if random_moving == True:
        ui_client("stop")
        cancelled_targets=cancelled_targets+1
        task_time=0
    else:
        random_moving = False
        msg.linear.x = setting_linear
        msg.angular.z = -setting_angular
        pub.publish(msg)
        print("Robot will go left-side after the command")

        

# Callback definition
start.on_click(start_robot)
stop.on_click(stop_robot)
forward.on_click(move_forward)
backward.on_click(move_backward)
right.on_click(turn_right)
left.on_click(turn_left)


# Buttons display
buttons = GridBox(children=[start, stop],layout=Layout(width='30%',grid_template_rows='auto auto',grid_template_columns='50% 50%',grid_template_areas='''"Start Stop up left right down"'''))
widgets.VBox([buttons, output])

buttons1 = VBox(children=[forward, left, right, backward],layout=Layout(width='30%',grid_template_rows='auto auto',grid_template_columns='20% 25%',grid_template_areas='''" . forward . ""left  right "".stop ."'''))
widgets.GridBox([buttons,buttons1, output])


GridBox(children=(GridBox(children=(Button(description='Start', layout=Layout(grid_area='Start', width='auto')…

# VISUALIZE ROBOT'S POSITION
* This cell will display the robot postion in the gazebo environment.robot's position is visualized using a black diamond shaped dots.the robot position is obtained by subscribing to /Odometry topic.

In [3]:
#plot for robot postion
fig_robot_position, ax_robot_position = plot.subplots()
ax_robot_position.set_xlim(( -15, 15))
ax_robot_position.set_ylim((-15, 15))
plot.xlabel("X axis")
plot.ylabel("Y axis")
plot.title("Robot moving positon")

line, = ax_robot_position.plot([], [], 'kd', markersize="6")

pos_x_data =  collections.deque(maxlen=300)
pos_y_data = collections.deque(maxlen=300)


def init():
    line.set_data([], [])
    return (line,)
    
def pos_animate(i):
    line.set_data(pos_x_data, pos_y_data)
    return (line)

def odometry_cb(msg):
    pos_y_data.append(msg.pose.pose.position.y)
    pos_x_data.append(msg.pose.pose.position.x) 
      
                  
# Read robot odometry
jr.subscribe('/odom', Odometry, odometry_cb)


anim_robot_pos = animation.FuncAnimation(fig_robot_position, pos_animate, init_func=init, blit=True)
plot.grid(b=True,color='r')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# VELOCITY VISUALIZATION
* Four line plots which will show the linear and angular velocity from cmd_vel message with respect to the actual angular and linear velocity from odom messages.

In [4]:
fig_linear_cmd_velocity,ax_linear_cmd_velocity=plot.subplots()
ax_linear_cmd_velocity.set_xlim(20, 0)
ax_linear_cmd_velocity.set_ylim(-1, 6)

fig_linear_actual_velocity,ax_linear_actual_velocity=plot.subplots()
ax_linear_actual_velocity.set_xlim(20, 0)
ax_linear_actual_velocity.set_ylim(-1, 6)

fig_angular_cmd_velocity,ax_angular_cmd_velocity=plot.subplots()
ax_angular_cmd_velocity.set_xlim(20, 0)
ax_angular_cmd_velocity.set_ylim(-1, 6)

fig_angular_actual_velocity,ax_angular_actual_velocity=plot.subplots()
ax_angular_actual_velocity.set_xlim(20, 0)
ax_angular_actual_velocity.set_ylim(-1, 6)


    # linear velocity
linear_cmd, = ax_linear_cmd_velocity.plot([], [], 'og', markersize='5')
linear_actual, = ax_linear_actual_velocity.plot([], [], 'ok', markersize='5')
    # angular velocity
angular_cmd, = ax_angular_cmd_velocity.plot([], [], 'oc', markersize='5')
angular_actual, = ax_angular_actual_velocity.plot([], [], 'om', markersize='5')

time_vect = collections.deque(maxlen=400)
vel_target = collections.deque(maxlen=400)
vel_current = collections.deque(maxlen=400)
angular_target = collections.deque(maxlen=400)
angular_current = collections.deque(maxlen=400)

def init_graph():
    # linear velocity lines
    linear_cmd.set_data([], [])
    linear_actual.set_data([], [])    
    # angular velocity lines
    angular_cmd.set_data([], [])
    angular_actual.set_data([], [])
    ax_linear_cmd_velocity.set_ylim(-5,5)
    ax_linear_actual_velocity.set_ylim(-5,5)
    ax_angular_cmd_velocity.set_ylim(-5,5)
    ax_angular_actual_velocity.set_ylim(-5,5)
    
    #return (line_vel_current, line_vel_target, ang_vel_current, ang_vel_target,)  
    return (linear_cmd, angular_cmd, linear_actual, angular_actual)
def animate_vel(i):   
    ax_linear_cmd_velocity.legend()
    ax_linear_cmd_velocity.set_title("cmd_vel linear velocity")
    ax_linear_cmd_velocity.set(xlabel='Time', ylabel='Linear velocity')
    ax_linear_cmd_velocity.set_xlabel('time')
    
    ax_linear_actual_velocity.legend()
    ax_linear_actual_velocity.set_title("odom linear velocity")
    ax_linear_actual_velocity.set(xlabel='Time', ylabel='Linear velocity')
    ax_linear_actual_velocity.set_xlabel('time')
    
    ax_angular_cmd_velocity.legend()
    ax_angular_cmd_velocity.set_title("cmd_vel angular velocity")
    ax_angular_cmd_velocity.set(xlabel='Time', ylabel='Angular velocity')
    ax_angular_cmd_velocity.set_xlabel('time')
    
    ax_angular_actual_velocity.legend()
    ax_angular_actual_velocity.set_title("odom angular velocity")
    ax_angular_actual_velocity.set(xlabel='Time', ylabel='Angular velocity')
    ax_angular_actual_velocity.set_xlabel('time')
    
    # linear velocity lines
    linear_cmd.set_data(time_vect, vel_target)   
    linear_actual.set_data(time_vect, vel_current)  
    # angular velocity lines
    angular_cmd.set_data(time_vect, angular_target) 
    angular_actual.set_data(time_vect, angular_current) 
    
    return (linear_cmd, angular_cmd, linear_actual, angular_actual)


def vel_callback(msg_cmd):   
    
    time_vect.append(rospy.get_time())       
    vel_target.append(msg_cmd.linear.x)  
    angular_target.append(msg_cmd.angular.z)  
    x_min = max([0, max(time_vect) - 20]) 
    x_max = max([100, max(time_vect)])
    ax_linear_cmd_velocity.set_xlim( x_min, x_max )
    ax_linear_actual_velocity.set_xlim( x_min, x_max )
    ax_angular_cmd_velocity.set_xlim( x_min, x_max )
    ax_angular_actual_velocity.set_xlim( x_min, x_max )
    
    
def odom_speed_callback(msg_odom):       
    vel_current.append(msg_odom.twist.twist.linear.x) 
    angular_current.append(-1.85*msg_odom.twist.twist.angular.z)

# Read robot velocity(cmd_vel)
jr.subscribe('cmd_vel', Twist, vel_callback)
# Read robot odometry(odom)
jr.subscribe('odom', Odometry, odom_speed_callback)

# Animation function for velocity check

anim_linear_cmd_velocity= animation.FuncAnimation(fig_linear_cmd_velocity, animate_vel, init_func=init_graph, frames=100, interval=20,
blit=True)

anim_angular_cmd_velocity= animation.FuncAnimation(fig_angular_cmd_velocity, animate_vel, init_func=init_graph, frames=100, interval=20,
blit=True)
anim_linear_actual_velocity= animation.FuncAnimation(fig_linear_actual_velocity, animate_vel, init_func=init_graph, frames=100, interval=20,
blit=True)

anim_angular_actual_velocity= animation.FuncAnimation(fig_angular_actual_velocity, animate_vel, init_func=init_graph, frames=100, interval=20,
blit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# HISTOGRAM:TIME TAKEN TO REACH A TARGET 
* Histogram plot in this block is used to show the time rached by the robot to reach a goal during random moving behaviour.
* A subscriber to "go_to_point/result" from the action "Planning" is used to know the time period.and the same process is used for the bar plot also.

In [5]:
fig_time_target,ax_time_target= plot.subplots() 

bins=15

ax_time_target.set_title("Time-goal")
ax_time_target.set(xlabel="Time periods", ylabel="Number of goal reached")
ax_time_target.hist([], bins)

def time_targets(msg):
  global reached_targets, random_moving, task_time, time_updation, ax_time_target, xlim
  if random_moving==True:
    reached_targets = reached_targets+1
    

    time_updation.append(rospy.Time.now().to_sec()-task_time.to_sec())
    task_time=rospy.Time.now()
    
        
    ax_time_target.hist(time_updation, bins, color='g')
  

jr.subscribe('/go_to_point/result',rt2_assignment1.msg.PlanningAction, time_targets)
fig_time_target.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# BAR PLOT:REACHED TARGETS VS CANCELLED TARGETS
* In this block, bar plot is used to show the number of reached targets and cancelled targets during the random movement behaviour .when the robot sucessfully reach a goal in its random movement behaviour, reached target will increase if the user cancels the random movement behaviuor,then the cancelled target will increase.

In [6]:
targets_fig=plot.figure()
targets_ax=targets_fig.add_subplot(111)
targets=[]
langs = ['Reached targets', 'Cacelled targets']

##x-axis indicates time and y axis indicates number of targets

def animate_targets(i):
    global reached_targets, cancelled_targets, targets
    targets=[reached_targets, cancelled_targets]
    targets_ax.bar(langs, targets, color='m')
    targets_ax.set_title("Reached targets vs Cancelled targets")
    if reached_targets>=cancelled_targets:
        targets_ax.set_ylim(0, reached_targets+10)
    else:
        targets_ax.set_ylim(0, cancelled_targets+10)
   
    

targets_bar = animation.FuncAnimation(targets_fig, animate_targets, interval=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …